In [1]:

%cd /home/teresakim/fungiclef-2024/
from fungiclef.utils import get_spark

# https://knowledge.informatica.com/s/article/000196886?language=en_US
# The vectorized reader will run out of memory (8gb) with the default batch size, so
# this is one way of handling the issue. This is likely due to the fact that the data
# column is so damn big, and treated as binary data instead of something like a string.
# We might also be able to avoid this if we don't cache the fields into memory, but this
# this needs to be validated by hand. 
spark = get_spark(**{
    # "spark.sql.parquet.columnarReaderBatchSize": 512,
    "spark.sql.parquet.enableVectorizedReader": False, 
})

gcs_parquet_path = "gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px/"

df = spark.read.parquet(gcs_parquet_path)
# df.printSchema()
# df.show(1, vertical=True, truncate=False)
# df.count() # 295938

/home/teresakim/fungiclef-2024


/home/fungi/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/29 06:26:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/02/29 06:26:52 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import torch

# load pretrained model
model = models.resnet18(pretrained=True).eval()

/home/fungi/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/fungi/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
# transformation for image preprocessing
transform = transforms.Compose([
    transforms.Resize(256), # 256x256 px
    transforms.CenterCrop(224), # going with common crop size, haven't messed with it
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [4]:
# UDF to convert binary image data to embedding
def binary_image_to_embedding(image_data):
    img = Image.open(io.BytesIO(image_data))
    img_t = transform(img)
    batch_t = torch.unsqueeze(img_t, 0)
    with torch.no_grad():
        embedding = model(batch_t)
   
    return embedding.cpu().numpy().flatten().tolist()

In [5]:
# register UDF with the appropriate return type
binary_to_embedding_udf = udf(binary_image_to_embedding, ArrayType(FloatType()))

In [6]:
# apply UDF
df_with_embeddings = df.withColumn("embeddings", binary_to_embedding_udf(df["data"]))


In [13]:
df_with_embeddings.show()

gcs_embedding_path = "gs://dsgt-clef-fungiclef-2024/data/parquet/DF20_300px_resnet18/"

df_with_embeddings.write.parquet(gcs_embedding_path)

24/02/29 06:44:51 ERROR Utils: Aborting task                       (0 + 4) / 65]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/tmp/ipykernel_10590/2562079121.py", line 3, in binary_image_to_embedding
NameError: name 'io' is not defined

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at org.apache.spark.InterruptibleIterator.hasNext(InterruptibleIterator.scala:37)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:491)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at org.apache.spark.sql.cat

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/tmp/ipykernel_10590/2562079121.py", line 3, in binary_image_to_embedding
NameError: name 'io' is not defined
